In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
test_config = {
    "image_size": 224,
    "feature_size": 56,
    "r_far": np.sqrt(0.5*0.5 + 0.5*0.5),
    "r_near": np.sqrt(0.5*0.5 + 0.5*0.5)
}

# feature map of 56x56 would give 3136 sources which
# would be enough. Otherwise maybe 28x28 depends on 
# images input size as what really matters is density of gal

In [3]:
def get_anchors(config):
    """
    Generates a lattice of anchors for a given image input
    size and feature map size. Note the coordinates are 
    (x,y) indexed!
    
    Adapted from Duncan Tilley's work on PPN 
    https://github.com/tilleyd/point-proposal-net/
    
    config
        The configuration dictionary.

    Returns the coorinates of the origin anchors (x,y).
    
    """
    
    # Reads sizes
    img_size = config['image_size']
    feature_size = config['feature_size']
    
    # Calculates step length for gievn sizes
    step = img_size / feature_size
    halfstep = step * 0.5
    
    # Calculates the lattice of points
    x = np.arange(halfstep, img_size, step, dtype=np.float32)
    y = np.arange(halfstep, img_size, step, dtype=np.float32)
    
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])


In [4]:
# testing get_anchors
print(get_anchors(test_config))

[[  2.   2.]
 [  6.   2.]
 [ 10.   2.]
 ...
 [214. 222.]
 [218. 222.]
 [222. 222.]]


In [7]:
def get_anchor_labels(anchors, truth_coords, config):
    """
    Creates anchor labels to be used in training.
    
    Adapted from Duncan Tilley's work on PPN 
    https://github.com/tilleyd/point-proposal-net/
    
    anchors
        The list of anchor coordinates generated from 
        get_anchors()
    
    truth_coords
        The list of ground truth point cooridnates.
        !!!check (x,y) or (y,x) convention
    
    config
        The configuration dictionary.
    
    Returns y_conf, y_reg
    
    """

    r_near = config['r_near']
    r_far = config['r_far']
    img_size = config['image_size']
    feature_size = config['feature_size']
    
    step = img_size / feature_size
    halfstep = step * 0.5

    # Initialising output
    y_conf = np.full(anchors.shape[0], -1, dtype=np.int8)
    y_reg = np.zeros(anchors.shape)
    
    # For each true point, find the nearest anchor
    for (x, y) in truth_coords:
        
        # Normalising truth points to step size
        x_norm = (x - halfstep) / step
        y_norm = (y - halfstep) / step
        
        # Finding index of closest anchor by rounding
        r = int(np.round(y_norm))
        c = int(np.round(x_norm))   
        anchor_index = r * feature_size + c
        
        # Setting values for anchor index
        y_conf[anchor_index] = 1
        y_reg[anchor_index][0] = (x - anchors[anchor_index][0]) / step
        y_reg[anchor_index][1] = (y - anchors[anchor_index][1]) / step
    
    # For each anchor calculate the distance to each point
    for i in range(len(anchors)):
        x, y = anchors[i]
        x /= step
        y /= step
        distances = []
        for (px, py) in truth_coords:
            px /= step
            py /= step
            distances.append(np.sqrt((x-px)**2 + (y-py)**2))
        if len(distances) > 0:
            near = np.argmin(distances)
            dist = distances[near]
            if dist <= r_near:
                y_conf[i] = 1
                px, py = truth_coords[near]
                px /= step
                py /= step
                y_reg[i][0] = (px - x)
                y_reg[i][1] = (py - y)
            elif dist > r_far:
                y_conf[i] = 0
                
    # reshape for use in PPN training
    y_conf = np.reshape(y_conf, (feature_size, feature_size))
    y_reg = np.reshape(y_reg, (feature_size, feature_size) + (2,))
    return y_conf, y_reg
    

try to avoid sqrt calculations as slow, could compare non-rooted values then just root the required value.
https://stackoverflow.com/questions/62400420/given-two-lists-of-2d-points-how-to-find-the-closest-point-in-the-2nd-list-for

speed here isn't really an issue so wouldn't worry about this.

In [17]:
#gt = np.array([[10,10],[100,50],[200,200]])
gt = np.random.randint(0,224,size=(1000,2))
anchs = get_anchors(test_config)
x = get_anchor_labels(anchs, gt, test_config)
print(gt.shape)

(1000, 2)


In [15]:
gt1 = np.random.randint(0,224,size=(2,447)) #define as (y,x) 
# Need to transform to shape (447,2) with first collumn x second y



print(gt1.shape)

(2, 447)


In [292]:
print(x[1])

[[[ 0.    0.  ]
  [ 0.    0.  ]
  [ 0.    0.  ]
  ...
  [-0.25 -0.5 ]
  [ 0.    0.  ]
  [-0.25  0.  ]]

 [[ 0.    0.  ]
  [ 0.    0.  ]
  [ 0.5  -0.25]
  ...
  [ 0.    0.  ]
  [ 0.    0.  ]
  [ 0.    0.  ]]

 [[ 0.    0.  ]
  [ 0.    0.  ]
  [ 0.    0.  ]
  ...
  [ 0.25  0.  ]
  [ 0.    0.  ]
  [ 0.    0.  ]]

 ...

 [[ 0.    0.  ]
  [ 0.    0.  ]
  [ 0.    0.  ]
  ...
  [-0.25  0.5 ]
  [ 0.    0.  ]
  [ 0.    0.  ]]

 [[ 0.25 -0.25]
  [ 0.    0.  ]
  [ 0.25  0.25]
  ...
  [ 0.    0.25]
  [ 0.    0.  ]
  [ 0.    0.  ]]

 [[-0.5  -0.25]
  [ 0.    0.  ]
  [ 0.    0.  ]
  ...
  [ 0.    0.  ]
  [ 0.    0.  ]
  [ 0.    0.  ]]]
